In [1]:
import pandas as pd
import yfinance as yf
from datetime import date, timedelta
from matplotlib import pyplot as plt
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests

C:\Users\ashkm\AppData\Local\Temp\ipykernel_49460\429619021.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#use to to look at the data for 1 ticker
ticker = yf.Ticker('JPY=X')
df = ticker.history(period = '6mo', interval='1d')
df.head()

C:\Users\ashkm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-08-28 00:00:00+01:00,146.531006,146.716003,146.278000,146.531006,0,0.0,0.0
2023-08-29 00:00:00+01:00,146.464005,147.343994,145.852005,146.464005,0,0.0,0.0
2023-08-30 00:00:00+01:00,145.988007,146.522995,145.557999,145.988007,0,0.0,0.0
2023-08-31 00:00:00+01:00,146.098999,146.190994,145.531006,146.098999,0,0.0,0.0
2023-09-01 00:00:00+01:00,145.473007,146.227005,144.481995,145.473007,0,0.0,0.0


In [3]:
# function to format in a useable df, only keeping the date part of datetime to avoid time-caused errors
# also we found that from yfinanve the 'Date' column was actually an index, so converted it to a column
def get_stock_data_to_df(ticker):
    ticker = yf.Ticker(str(ticker))
    df = ticker.history(period = '6mo', interval='1d')
    df.reset_index(inplace=True)
    df["timestamp_processed"] = [e.strftime('%Y-%m-%d') for e in df["Date"]]
    return df

# making the data stionary to fit the assumption. logging the price for the column given and taking the diffence
# created a new column for that called 'stationary'
def stationaryize(df : pd.DataFrame, time) -> pd.DataFrame:
    df['stationary'] = (np.log(df[time])).diff()
    df = df.dropna(how = 'any', axis = 0, )
    return df


def get_clean_data(ticker, time) -> pd.DataFrame:
    return stationaryize(get_stock_data_to_df(ticker.upper()), time)


# main function here which takes in 2 tickers and the open/close/high/low column, and merges the stationary columnms for each on the timestamp column

def merge_df(ticker1, ticker2, open_or_close):
    df1 = get_clean_data(ticker1, open_or_close)
    # print("df of  " + ticker1 + " " + str(np.shape(df1)))
    # print("df of " + ticker1)
    # print(df1.head())
    df2 = get_clean_data(ticker2, open_or_close)
    # print("df of  " + ticker2 + " " + str(np.shape(df2)))
    # print("df of " + ticker2)
    # print(df2.head())
    # return pd.merge(df1[['stationary']], df2[['stationary']], left_index=True, right_index=True, how='right').rename(columns={'stationary_x': ticker1, 'stationary_y': ticker2}).dropna(how='any')
    return pd.merge(df1[['timestamp_processed', 'stationary']], df2[['timestamp_processed', 'stationary']], on='timestamp_processed', how='right').rename(columns = {'stationary_x': ticker1, 'stationary_y': ticker2}).dropna(how='any')


In [4]:
merge_df('^TNX', '^GSPC', 'Close')

C:\Users\ashkm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\ashkm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


,timestamp_processed,^TNX,^GSPC
0,2023-08-29,-0.021599,0.014404
1,2023-08-30,-0.000971,0.003826
2,2023-08-31,-0.006089,-0.001598
3,2023-09-01,0.019357,0.001798
4,2023-09-05,0.022510,-0.004203
...,...,...,...
119,2024-02-20,-0.004667,-0.006023
120,2024-02-21,0.011628,0.001263
121,2024-02-22,0.000462,0.020903
122,2024-02-23,-0.015605,0.000348


In [50]:
# function for the granger test. takes in the tickers, time(open/close/high/low), and lag, and calls merge_df function. returns the test


def granger_causality(ticker1, ticker2, time, lag):
    df = merge_df(ticker1, ticker2, time)
    # if len(df[ticker1]) != len(df[ticker2]):
    # minrows = min(len(df[ticker1]), len(df[ticker2]))
    # df = df.iloc[:,:minrows]
    #print(np.shape(df))
    return grangercausalitytests(df[[ticker1, ticker2]], lag)

In [51]:
granger_causality('TSLA', 'AAPL', 'Open', 4)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.1899  , p=0.2775  , df_denom=120, df_num=1
ssr based chi2 test:   chi2=1.2197  , p=0.2694  , df=1
likelihood ratio test: chi2=1.2137  , p=0.2706  , df=1
parameter F test:         F=1.1899  , p=0.2775  , df_denom=120, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.0251  , p=0.3619  , df_denom=117, df_num=2
ssr based chi2 test:   chi2=2.1379  , p=0.3434  , df=2
likelihood ratio test: chi2=2.1194  , p=0.3466  , df=2
parameter F test:         F=1.0251  , p=0.3619  , df_denom=117, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.4144  , p=0.7430  , df_denom=114, df_num=3
ssr based chi2 test:   chi2=1.3196  , p=0.7245  , df=3
likelihood ratio test: chi2=1.3124  , p=0.7262  , df=3
parameter F test:         F=0.4144  , p=0.7430  , df_denom=114, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.4832  , p=0.7480  

C:\Users\ashkm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\ashkm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


{1: ({'ssr_ftest': (1.1899273166759798, 0.27752969703960295, 120.0, 1),
   'ssr_chi2test': (1.2196754995928791, 0.2694243447484484, 1),
   'lrtest': (1.2136679925091585, 0.2706069001487983, 1),
   'params_ftest': (1.1899273166760072, 0.2775296970395981, 120.0, 1.0)},
   array([[0., 1., 0.]])]),
 2: ({'ssr_ftest': (1.0251472496757352, 0.3619429975769713, 117.0, 2),
   'ssr_chi2test': (2.1379139224006787, 0.3433664752604946, 2),
   'lrtest': (2.1193976490103523, 0.3465601700446738, 2),
   'params_ftest': (1.0251472496757386, 0.3619429975769689, 117.0, 2.0)},
   array([[0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]])]),
 3: ({'ssr_ftest': (0.41440806250747747, 0.742975234183745, 114.0, 3),
   'ssr_chi2test': (1.3195625148264412, 0.7244930885874101, 3),
   'lrtest': (1.312419174027582, 0.7261861543821118, 3),
   'params_ftest': (0.41440806250747725, 0.742975234183745, 114.0, 3.0)},
   array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 

In [52]:
granger_causality('^TNX', '^GSPC', 'Close', 4)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0640  , p=0.8007  , df_denom=120, df_num=1
ssr based chi2 test:   chi2=0.0656  , p=0.7978  , df=1
likelihood ratio test: chi2=0.0656  , p=0.7979  , df=1
parameter F test:         F=0.0640  , p=0.8007  , df_denom=120, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=3.2749  , p=0.0413  , df_denom=117, df_num=2
ssr based chi2 test:   chi2=6.8297  , p=0.0329  , df=2
likelihood ratio test: chi2=6.6454  , p=0.0361  , df=2
parameter F test:         F=3.2749  , p=0.0413  , df_denom=117, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=2.7076  , p=0.0485  , df_denom=114, df_num=3
ssr based chi2 test:   chi2=8.6215  , p=0.0348  , df=3
likelihood ratio test: chi2=8.3282  , p=0.0397  , df=3
parameter F test:         F=2.7076  , p=0.0485  , df_denom=114, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=2.0541  , p=0.0917  

C:\Users\ashkm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\ashkm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


{1: ({'ssr_ftest': (0.06401272682270631, 0.8006954335380561, 120.0, 1),
   'ssr_chi2test': (0.06561304499327397, 0.7978346797197604, 1),
   'lrtest': (0.06559555092303526, 0.797861048436457, 1),
   'params_ftest': (0.06401272682269271, 0.8006954335380764, 120.0, 1.0)},
   array([[0., 1., 0.]])]),
 2: ({'ssr_ftest': (3.274888057601406, 0.04131598309980075, 117.0, 2),
   'ssr_chi2test': (6.829681077390967, 0.03288164963422522, 2),
   'lrtest': (6.645362635452898, 0.036056024369081084, 2),
   'params_ftest': (3.2748880576014083, 0.04131598309980075, 117.0, 2.0)},
   array([[0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]])]),
 3: ({'ssr_ftest': (2.707566578832429, 0.04854785592024153, 114.0, 3),
   'ssr_chi2test': (8.621462001019049, 0.034771033172953866, 3),
   'lrtest': (8.328167002951659, 0.03969470479773607, 3),
   'params_ftest': (2.707566578832427, 0.04854785592024153, 114.0, 3.0)},
   array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0.],
          [0., 0.

In [ ]:
granger_causality("amx", 'aapl', 'Close', 7)

In [17]:
# def get_all_tickers(client) -> list[str]:
#     response = client.get_snapshot_all('stocks')
#     tickers = [i.ticker for i in response]
#     return tickers
# def get_test_results():
#     tickers = get_all_tickers(client)
#     all_correlations = [[-1 for i in range(len(tickers) - 1)] for j in range(len(tickers) - 1)]
#     all_trials = len(tickers) * (len(tickers) - 1) // 2
#     success = 0
#     for ticker1 in range(len(tickers) - 1):
#         for ticker2 in range(ticker1 + 1, len(tickers) - 1):
#             print(f'{success} / {all_trials} done')
#             t1 = tickers[ticker1]
#             t2 = tickers[ticker2]
#             try:
#                 merged = merge_df(t1, t2,'close')
#                 test = grangercausalitytests(merged[[t1, t2]], 7)
#                 best_corr = max(test, key = lambda x: x[1][0]['ssr_ftest'][1])
#                 success += 1
#             except:
#                 continue
#             all_correlations[ticker1][ticker2] = best_corr[0]
#     return all_correlations

In [18]:
# # %pip install matplotlib
# import matplotlib.pyplot as plt
# print("getting all data correlations")
# all_correlations = get_test_results()
# print("done getting all correlations")
# plt.imshow(a, cmap='hot', interpolation='nearest')
# plt.show()
# plt.savefig('./big_correlation_map')

getting all data correlations


NameError: name 'client' is not defined

In [19]:
ticker_lst = ['aapl', 'goog', 'amd', 'nvda', 'are', 'amx', 'cof', 'cvx', 'f', 'ma']

In [60]:
def get_matrix_data(tickers):
    correlations = [[-1 for i in range(len(tickers))] for j in range(len(tickers))]
    for i in range(len(tickers)):
        for j in range(len(tickers)):
            if i != j:
                correlations[i][j] = max(granger_causality(tickers[i], tickers[j], 'Close', 1).items(), key = lambda x: x[1])
                
    return correlations[0][1][1][0]['ssr_ftest'][1]        